In [1]:
import pandas as pd 
import numpy as np
import re

##LDA stuff
import gensim
from gensim.utils import simple_preprocess
import nltk
from nltk.corpus import words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

##cleaning stuff
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

import gensim.corpora as corpora

In [2]:
df = pd.read_csv('../data/cleaned/cleaned_ibis-sg-bencoolen.csv')
df.head()

,Unnamed: 0,date_of_stay,traveller_username,review_title,review_text,travel_type,traveller_country_origin,traveller_total_contributions,traveller_total_helpful_contributions,rating1,rating2,rating,valid_rating,label,cleaned_review,combined_review,date,covid
0,0,Date of stay: August 2023,Love_Life_Sydney,Clean and comfortable,Hotel rooms in Singapore are so expensive so t...,Trip type: Travelled as a couple,"Sydney, Australia",2302.0,871.0,NaN,4.0,4.0,True,Positive,clean comfortable hotel room singapore expensi...,Clean and comfortable Hotel rooms in Singapore...,2023-08-01,PostCovid
1,1,Date of stay: August 2023,Bilal S,"Good hotel, great location",This is a great place! Location is great but t...,Trip type: Travelled with family,"Houston, Texas",4.0,NaN,5.0,NaN,5.0,True,Positive,good hotel great location great place location...,"Good hotel, great location This is a great pl...",2023-08-01,PostCovid
2,2,Date of stay: October 2022,Anthony Fernando,Good place for a decent price.,Good place good price Easy access to the city...,Trip type: Travelled with friends,"Dubai, United Arab Emirates",39.0,38.0,5.0,NaN,5.0,True,Positive,good place decent price good place good price ...,Good place for a decent price. Good place good...,2022-10-01,PostCovid
3,3,Date of stay: August 2023,Mjkc204,Great Location and great staff.,The IBIS was a neat and tidy hotel in line wit...,Trip type: Travelled solo,"Ellenbrook, Australia",37.0,19.0,5.0,NaN,5.0,True,Positive,great location great staff ibis neat tidy hote...,Great Location and great staff. The IBIS was a...,2023-08-01,PostCovid
4,4,Date of stay: August 2022,Aung Nanda,Good for budget stay.,I stayed there for 7 days. It was a nice locat...,Trip type: Travelled on business,"Dubai, United Arab Emirates",3.0,4.0,NaN,4.0,4.0,True,Positive,good budget stay I stay day nice location seve...,Good for budget stay. I stayed there for 7 day...,2022-08-01,PostCovid


In [5]:
##filter for year >= 2015
df['date_of_stay'] = pd.to_datetime(df['date_of_stay'], format='Date of stay: %B %Y')
df_filtered = df[df['date_of_stay'].dt.year >= 2015]

##preprocess text
nltk.download('stopwords')
nltk.download('words')
# Get the list of valid English words
english_words = set(words.words())
# set stopwords
sw = stopwords.words('english')
sw.append('ibis')
stop_words = set(sw)
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to C:\Users\Chen
[nltk_data]     Wei\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to C:\Users\Chen
[nltk_data]     Wei\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [8]:
def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word.isalnum()]
    tokens = [word.lower() for word in tokens]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

In [9]:
def remove_non_english_words(text, valid_words):
    tokens = word_tokenize(text)
    ans = [w for w in tokens if w.lower() in valid_words]
    return ' '.join(ans)

def remove_stopwords(text):
    tokens = word_tokenize(text)
    ans = [w for w in tokens if w.lower() not in stop_words]
    return ' '.join(ans)

In [10]:
df_filtered['cleaned_review'] = df_filtered['cleaned_review'].apply(remove_stopwords)
df_filtered['cleaned_review'] = df_filtered['cleaned_review'].apply(remove_non_english_words, valid_words=english_words)
df_filtered['tokens'] = df_filtered['cleaned_review'].apply(preprocess_text)

C:\Users\Chen Wei\AppData\Local\Temp\ipykernel_10868\2710312924.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['cleaned_review'] = df_filtered['cleaned_review'].apply(remove_stopwords)
C:\Users\Chen Wei\AppData\Local\Temp\ipykernel_10868\2710312924.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['cleaned_review'] = df_filtered['cleaned_review'].apply(remove_non_english_words, valid_words=english_words)
C:\Users\Chen Wei\AppData\Local\Temp\ipykernel_10868\2710312924.py:3: Set

In [11]:
df_precovid = df_filtered[df_filtered['covid'] == 'PreCovid']
df_precovid.head()

,Unnamed: 0,date_of_stay,traveller_username,review_title,review_text,travel_type,traveller_country_origin,traveller_total_contributions,traveller_total_helpful_contributions,rating1,rating2,rating,valid_rating,label,cleaned_review,combined_review,date,covid,tokens
66,66,2019-12-01,Beatrice,Cozy Hotel,I spent 2 night at this hotel and all the serv...,NaN,"Batam, Indonesia",8.0,13.0,5.0,NaN,5.0,True,Positive,cozy hotel spend night hotel service provide h...,Cozy Hotel I spent 2 night at this hotel and a...,2019-12-01,PreCovid,"[cozy, hotel, spend, night, hotel, service, pr..."
67,67,2020-01-01,dinu79,"Smart, Clean, Good Location, Value for money",This was my second stay at Ibis Bencoolen. B...,NaN,"Dubai, United Arab Emirates",212.0,53.0,4.0,NaN,4.0,True,Positive,smart clean good location value money second s...,"Smart, Clean, Good Location, Value for money T...",2020-01-01,PreCovid,"[smart, clean, good, location, value, money, s..."
68,68,2019-10-01,Marsha F,Wonderful experience,"This was our first time at this hotel, and won...",NaN,"Gatineau, Canada",32.0,45.0,4.0,NaN,4.0,True,Positive,wonderful experience first time hotel win last...,Wonderful experience This was our first time a...,2019-10-01,PreCovid,"[wonderful, experience, first, time, hotel, wi..."
69,69,2019-10-01,Geoff_and_Juls,Great Base for Tourists,"Firstly, the good points: Comfortable and clea...",NaN,"Mellieha, Malta",276.0,169.0,4.0,NaN,4.0,True,Positive,great base tourist firstly good point comforta...,"Great Base for Tourists Firstly, the good poin...",2019-10-01,PreCovid,"[great, base, tourist, firstly, good, point, c..."
70,70,2019-10-01,Gianluca3008,BEST LOCATION,"Very comfortable hotel, nice staff, totally in...",NaN,"Rome, Italy",2192.0,280.0,NaN,4.0,4.0,True,Positive,good location comfortable hotel nice staff tot...,"BEST LOCATION Very comfortable hotel, nice sta...",2019-10-01,PreCovid,"[good, location, comfortable, hotel, nice, sta..."


In [12]:
df_postcovid = df_filtered[df_filtered['covid'] == 'PostCovid']
df_postcovid.head()

,Unnamed: 0,date_of_stay,traveller_username,review_title,review_text,travel_type,traveller_country_origin,traveller_total_contributions,traveller_total_helpful_contributions,rating1,rating2,rating,valid_rating,label,cleaned_review,combined_review,date,covid,tokens
0,0,2023-08-01,Love_Life_Sydney,Clean and comfortable,Hotel rooms in Singapore are so expensive so t...,Trip type: Travelled as a couple,"Sydney, Australia",2302.0,871.0,NaN,4.0,4.0,True,Positive,clean comfortable hotel room expensive find de...,Clean and comfortable Hotel rooms in Singapore...,2023-08-01,PostCovid,"[clean, comfortable, hotel, room, expensive, f..."
1,1,2023-08-01,Bilal S,"Good hotel, great location",This is a great place! Location is great but t...,Trip type: Travelled with family,"Houston, Texas",4.0,NaN,5.0,NaN,5.0,True,Positive,good hotel great location great place location...,"Good hotel, great location This is a great pl...",2023-08-01,PostCovid,"[good, hotel, great, location, great, place, l..."
2,2,2022-10-01,Anthony Fernando,Good place for a decent price.,Good place good price Easy access to the city...,Trip type: Travelled with friends,"Dubai, United Arab Emirates",39.0,38.0,5.0,NaN,5.0,True,Positive,good place decent price good place good price ...,Good place for a decent price. Good place good...,2022-10-01,PostCovid,"[good, place, decent, price, good, place, good..."
3,3,2023-08-01,Mjkc204,Great Location and great staff.,The IBIS was a neat and tidy hotel in line wit...,Trip type: Travelled solo,"Ellenbrook, Australia",37.0,19.0,5.0,NaN,5.0,True,Positive,great location great staff neat tidy hotel lin...,Great Location and great staff. The IBIS was a...,2023-08-01,PostCovid,"[great, location, great, staff, neat, tidy, ho..."
4,4,2022-08-01,Aung Nanda,Good for budget stay.,I stayed there for 7 days. It was a nice locat...,Trip type: Travelled on business,"Dubai, United Arab Emirates",3.0,4.0,NaN,4.0,4.0,True,Positive,good budget stay stay day nice location seven ...,Good for budget stay. I stayed there for 7 day...,2022-08-01,PostCovid,"[good, budget, stay, stay, day, nice, location..."


## Unigrams Cosine Similarity

### Get top 10 for topics

In [13]:
##Generate LDA dictionary and corpus
pre_dict = corpora.Dictionary(df_precovid['tokens'])
pre_corpus = [pre_dict.doc2bow(text) for text in df_precovid['tokens']]

In [14]:
## Check optimal number of topics
lda = gensim.models.ldamodel.LdaModel
pre_ldamodel = lda(pre_corpus, num_topics=5, id2word=pre_dict, passes=15)

In [16]:
# Show topics with different relevance values
# For example, set topn=10 to display the top 10 terms per topic
topics_with_high_relevance = pre_ldamodel.show_topics(num_topics=5, num_words=10)
topics_with_low_relevance = pre_ldamodel.show_topics(num_topics=5, num_words=5)  # Lower relevance

# Print and examine the topics
print("Topics with high relevance:")
for topic_id, topic in topics_with_high_relevance:
    print(f"Topic {topic_id}: {topic}")

print("\nTopics with low relevance:")
for topic_id, topic in topics_with_low_relevance:
    print(f"Topic {topic_id}: {topic}")

Topics with high relevance:
Topic 0: 0.026*"hotel" + 0.023*"shower" + 0.019*"food" + 0.015*"also" + 0.012*"use" + 0.011*"hot" + 0.011*"water" + 0.009*"area" + 0.009*"court" + 0.008*"however"
Topic 1: 0.040*"room" + 0.020*"hotel" + 0.017*"stay" + 0.012*"good" + 0.012*"bed" + 0.012*"water" + 0.011*"staff" + 0.010*"one" + 0.010*"free" + 0.010*"service"
Topic 2: 0.036*"team" + 0.022*"stay" + 0.020*"hotel" + 0.016*"answer" + 0.016*"care" + 0.015*"matter" + 0.014*"reservation" + 0.013*"customer" + 0.013*"one" + 0.012*"much"
Topic 3: 0.041*"hotel" + 0.030*"room" + 0.026*"stay" + 0.023*"good" + 0.018*"great" + 0.017*"location" + 0.012*"staff" + 0.012*"walk" + 0.011*"clean" + 0.011*"check"
Topic 4: 0.048*"hotel" + 0.035*"good" + 0.025*"room" + 0.025*"location" + 0.019*"clean" + 0.017*"staff" + 0.017*"stay" + 0.014*"nice" + 0.012*"near" + 0.011*"friendly"

Topics with low relevance:
Topic 0: 0.026*"hotel" + 0.023*"shower" + 0.019*"food" + 0.015*"also" + 0.012*"use"
Topic 1: 0.040*"room" + 0.020*

In [30]:
topic_words = []
for i in range(5):
    top_10_ids = pre_ldamodel.get_topic_terms(i, 10)
    top_words = [pre_dict[ind] for ind, val in top_10_ids]
    topic_words.append(top_words)


In [31]:
topic_words

[['hotel',
  'shower',
  'food',
  'also',
  'use',
  'hot',
  'water',
  'area',
  'court',
  'however'],
 ['room',
  'hotel',
  'stay',
  'good',
  'bed',
  'water',
  'staff',
  'one',
  'free',
  'service'],
 ['team',
  'stay',
  'hotel',
  'answer',
  'care',
  'matter',
  'reservation',
  'customer',
  'one',
  'much'],
 ['hotel',
  'room',
  'stay',
  'good',
  'great',
  'location',
  'staff',
  'walk',
  'clean',
  'check'],
 ['hotel',
  'good',
  'room',
  'location',
  'clean',
  'staff',
  'stay',
  'nice',
  'near',
  'friendly']]

In [96]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
import scipy.sparse

In [39]:
df_precovid['vec_tokens'] = df_precovid['tokens'].apply(lambda x: ' '.join(x))

C:\Users\Chen Wei\AppData\Local\Temp\ipykernel_10868\1014971942.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_precovid['vec_tokens'] = df_precovid['tokens'].apply(lambda x: ' '.join(x))


In [144]:
df_precovid['vec_tokens']

66      cozy hotel spend night hotel service provide h...
67      smart clean good location value money second s...
68      wonderful experience first time hotel win last...
69      great base tourist firstly good point comforta...
70      good location comfortable hotel nice staff tot...
                              ...                        
2274    perfect location lovely two day stop complemen...
2275    great hotel place perfect day reasonably good ...
2277    budget hotel good location partner stay part l...
2280    convenient location clean room friendly staff ...
2286    perfect hotel fantastic give expensive country...
Name: vec_tokens, Length: 2141, dtype: object

In [44]:
vectorizer = CountVectorizer()
transformer = TfidfTransformer()

precovid_vec = vectorizer.fit_transform(df_precovid['vec_tokens'])
precovid_trf = transformer.fit_transform(precovid_vec)

In [82]:
topic_words[0]

['hotel',
 'shower',
 'food',
 'also',
 'use',
 'hot',
 'water',
 'area',
 'court',
 'however']

In [159]:
cos_res = {}
for i in range(5):
    topic_words_vec = vectorizer.transform(topic_words[i])
    topic_words_trf = transformer.transform(topic_words_vec)
    r = cosine_similarity(precovid_trf, topic_words_trf)
    cos_sim = np.rad2deg(np.arccos(r))
    
    mid_res = []
    for j in cos_sim:
        average_cos_sim = ((j == 90).sum()/len(j))
        mid_res.append(average_cos_sim)
    cos_res[i] = mid_res

In [173]:
df_c = df_precovid.reset_index()

In [188]:
sentences_used = set()
for i in cos_res:
    print(f"Sentences for topic {i}:")
    for j in range(len(cos_res[i])):
        if cos_res[i][j] < 0.6:
            print(f"Sentence: {j}:")
            sentences_used.add(j)
            print(df_c.loc[j, 'vec_tokens'])
            

Sentences for topic 0:
Sentence: 24:
budget friendly superb location hotel room fuss free clean plenty affordable food around hotel student area away bus stop right front hotel also provide self service laundry room refillable hot cold water dispenser come certainly stay
Sentence: 44:
fantastic hotel great price food school stay hotel international conference attend actually never stay first time stay hotel first foremost room great somewhat small decent sized room ample usual amenity coat hanger iron board desk outlet hotel phone bottled water everything else bed comfortable like hotel bed allow full night rest room great well every time room get clean well bathroom room great well spacious hot water water pressure perfect furthermore hotel weak time sometimes
Sentence: 47:
excellent hotel time come airport allow check instantly without extra charge awesome stay king size bed room near min walk little hotel also bus stop great location hotel service good lot food court min walk distan

In [189]:
unused_sentences = []
for i in range(len(df_c)):
    if i not in sentences_used:
        unused_sentences.append(i)

print(len(unused_sentences))

222


## Bigram Cosine Similarity

In [190]:
clean_precovid = df_precovid['cleaned_review'].apply(preprocess_text)
clean_precovid = clean_precovid.apply(lambda x: ' '.join(x))
clean_postcovid = df_postcovid['cleaned_review'].apply(preprocess_text)
clean_postcovid = clean_postcovid.apply(lambda x: ' '.join(x))

In [191]:
from nltk import ngrams
bigrams_precovid = []
bigrams_postcovid = []

for d in clean_precovid:
    tokens = word_tokenize(d)
    bigrams_precovid.append(["_".join(w) for w in ngrams(tokens, 2)])

for e in clean_postcovid:
    tokens = word_tokenize(e)
    bigrams_postcovid.append(['_'.join(w) for w in ngrams(tokens, 2)])


In [192]:
##Generate LDA dictionary and corpus
pre_dict = corpora.Dictionary(bigrams_precovid)
pre_corpus = [pre_dict.doc2bow(text) for text in bigrams_precovid]

In [193]:
## Check optimal number of topics
lda = gensim.models.ldamodel.LdaModel
pre_ldamodel = lda(pre_corpus, num_topics=5, id2word=pre_dict, passes=15)

In [194]:
##Generate LDA dictionary and corpus
post_dict = corpora.Dictionary(bigrams_postcovid)
post_corpus = [post_dict.doc2bow(text) for text in bigrams_postcovid]

In [195]:
## Check optimal number of topics
lda = gensim.models.ldamodel.LdaModel
post_ldamodel = lda(post_corpus, num_topics=5, id2word=post_dict, passes=15)

In [196]:
# Show topics with different relevance values
# For example, set topn=10 to display the top 10 terms per topic
topics_with_high_relevance = post_ldamodel.show_topics(num_topics=4, num_words=10)
topics_with_low_relevance = post_ldamodel.show_topics(num_topics=4, num_words=5)  # Lower relevance

# Print and examine the topics
print("Topics with high relevance:")
for topic_id, topic in topics_with_high_relevance:
    print(f"Topic {topic_id}: {topic}")

print("\nTopics with low relevance:")
for topic_id, topic in topics_with_low_relevance:
    print(f"Topic {topic_id}: {topic}")

Topics with high relevance:
Topic 4: 0.004*"clean_room" + 0.003*"walk_distance" + 0.003*"come_back" + 0.003*"friendly_service" + 0.003*"customer_service" + 0.003*"definitely_stay" + 0.003*"would_definitely" + 0.003*"short_walk" + 0.002*"good_location" + 0.002*"next_door"
Topic 2: 0.005*"great_location" + 0.003*"hotel_great" + 0.003*"clean_hotel" + 0.003*"hotel_room" + 0.003*"cancel_booking" + 0.003*"stay_hotel" + 0.003*"room_good" + 0.003*"staff_friendly" + 0.003*"good_hotel" + 0.002*"next_door"
Topic 0: 0.003*"walk_distance" + 0.003*"engineering_team" + 0.003*"bed_comfortable" + 0.003*"small_room" + 0.003*"good_location" + 0.003*"cleaning_staff" + 0.003*"extra_benefit" + 0.003*"room_small" + 0.002*"good_place" + 0.002*"reservation_breakfast"
Topic 1: 0.006*"value_money" + 0.003*"location_great" + 0.003*"good_hotel" + 0.002*"bed_comfortable" + 0.002*"excellent_service" + 0.002*"great_location" + 0.002*"buffet_breakfast" + 0.002*"good_value" + 0.002*"check_good" + 0.002*"queue_check"

T

In [197]:
topic_words = []
for i in range(5):
    top_10_ids = pre_ldamodel.get_topic_terms(i, 10)
    top_words = [pre_dict[ind] for ind, val in top_10_ids]
    topic_words.append(top_words)


In [198]:
topic_words

[['stay_hotel',
  'room_clean',
  'good_location',
  'minute_walk',
  'great_location',
  'good_experience',
  'value_money',
  'come_back',
  'orchard_road',
  'room_comfortable'],
 ['good_location',
  'bus_stop',
  'stay_hotel',
  'room_clean',
  'great_location',
  'hotel_room',
  'good_hotel',
  'walk_distance',
  'location_stay',
  'minute_walk'],
 ['bus_stop',
  'good_location',
  'minute_walk',
  'front_hotel',
  'walk_distance',
  'staff_friendly',
  'room_clean',
  'great_hotel',
  'hotel_good',
  'great_location'],
 ['good_hotel',
  'value_money',
  'comfortable_bed',
  'minute_walk',
  'budget_hotel',
  'room_small',
  'staff_friendly',
  'bed_comfortable',
  'great_location',
  'good_location'],
 ['room_small',
  'good_location',
  'staff_friendly',
  'walk_distance',
  'good_value',
  'value_money',
  'great_location',
  'friendly_helpful',
  'room_clean',
  'bus_stop']]

In [202]:
df_precovid['vec_tokens'] = df_precovid['tokens'].apply(lambda x: ["_".join(w) for w in ngrams(x, 2)])

C:\Users\Chen Wei\AppData\Local\Temp\ipykernel_10868\2572606165.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_precovid['vec_tokens'] = df_precovid['tokens'].apply(lambda x: ["_".join(w) for w in ngrams(x, 2)])


In [205]:
df_precovid['vec'] = df_precovid['vec_tokens'].apply(lambda x: ' '.join(x))

C:\Users\Chen Wei\AppData\Local\Temp\ipykernel_10868\416102357.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_precovid['vec'] = df_precovid['vec_tokens'].apply(lambda x: ' '.join(x))


In [207]:
vectorizer = CountVectorizer()
transformer = TfidfTransformer()

precovid_vec = vectorizer.fit_transform(df_precovid['vec'])
precovid_trf = transformer.fit_transform(precovid_vec)

In [208]:
cos_res = {}
for i in range(5):
    topic_words_vec = vectorizer.transform(topic_words[i])
    topic_words_trf = transformer.transform(topic_words_vec)
    r = cosine_similarity(precovid_trf, topic_words_trf)
    cos_sim = np.rad2deg(np.arccos(r))
    
    mid_res = []
    for j in cos_sim:
        average_cos_sim = ((j == 90).sum()/len(j))
        mid_res.append(average_cos_sim)
    cos_res[i] = mid_res

In [209]:
df_c = df_precovid.reset_index()

In [212]:
sentences_used = set()
for i in cos_res:
    print(f"Sentences for topic {i}:")
    for j in range(len(cos_res[i])):
        if cos_res[i][j] < 0.6:
            print(f"Sentence: {j}:")
            sentences_used.add(j)
            print(df_c.loc[j, 'vec_tokens'])
            

Sentences for topic 0:
Sentence: 2130:
['stay_come', 'come_back', 'back_second', 'second_stay', 'stay_room', 'room_clean', 'clean_size', 'size_good', 'good_bed', 'bed_pillow', 'pillow_great', 'great_even', 'even_well', 'well_well', 'well_know', 'know_star', 'star_hotel', 'hotel_city', 'city_sleep', 'sleep_really', 'really_well', 'well_restaurant', 'restaurant_great', 'great_food', 'food_beverage', 'beverage_good', 'good_location', 'location_public', 'public_transport', 'transport_within', 'within_reach', 'reach_bus', 'bus_stop', 'stop_right', 'right_front', 'front_hotel', 'hotel_minute', 'minute_walk', 'walk_close', 'close_station', 'station_overall', 'overall_great', 'great_hotel', 'hotel_stay', 'stay_good', 'good_value', 'value_money', 'money_satisfied', 'satisfied_come', 'come_back', 'back_sure']
Sentences for topic 1:
Sentence: 1239:
['good_hotel', 'hotel_great', 'great_location', 'location_stay', 'stay_couple', 'couple_night', 'night_june', 'june_really', 'really_complaint', 'comp

In [213]:
unused_sentences = []
for i in range(len(df_c)):
    if i not in sentences_used:
        unused_sentences.append(i)

print(len(unused_sentences))

2120
